In [1]:
# coding: utf-8  
import numpy as np    
import random     
import matplotlib.pyplot as plt 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from imutils import paths
import imutils
import cv2
import os

import time


/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
trainImage_CNN = np.load('./trainImage_CNN.npy')
testImage_CNN = np.load('./testImage_CNN.npy')

trainLabels = np.load('./trainLabels.npy')
testLabels= np.load('./testLabels.npy')

n_feature = trainImage_CNN.shape[1]
gamma_init = 1/n_feature
print (trainImage_CNN.shape)
print (n_feature)
print (gamma_init)


(2000, 25088)
25088
3.985969387755102e-05


In [3]:
# SVC
print("\n")
print("[INFO] CNN accuracy...")
model_SVM = SVC(max_iter=200, class_weight='balanced' )
model_SVM.fit(trainImage_CNN, trainLabels)
acc = model_SVM.score(testImage_CNN, testLabels)
print("[INFO] SVM-SVC accuracy: {:.2f}%".format(acc * 100))



[INFO] CNN accuracy...


/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[INFO] SVM-SVC accuracy: 68.35%


In [4]:
# SVC
print("\n")
print("[INFO] CNN accuracy...")
model_SVM = SVC(max_iter=1000, class_weight='balanced' )
model_SVM.fit(trainImage_CNN, trainLabels)
acc = model_SVM.score(testImage_CNN, testLabels)
print("[INFO] SVM-SVC accuracy: {:.2f}%".format(acc * 100))



[INFO] CNN accuracy...
[INFO] SVM-SVC accuracy: 69.94%


In [5]:
# SVC
print("\n")
print("[INFO] CNN accuracy...")
model_SVM = SVC(max_iter=10000, class_weight='balanced' )
model_SVM.fit(trainImage_CNN, trainLabels)
acc = model_SVM.score(testImage_CNN, testLabels)
print("[INFO] SVM-SVC accuracy: {:.2f}%".format(acc * 100))



[INFO] CNN accuracy...
[INFO] SVM-SVC accuracy: 69.94%


In [6]:
model_SVM

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=10000, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [7]:
#----------------------PSO参数设置---------------------------------    
class PSO():    
    def __init__(self,pN,max_iter):    
        self.w = 0.8      
        self.c1 = 1  
        self.r1= 0.6 
        self.c2 = 1       
        self.r2= 0.3    
        self.pN = pN                #粒子数量    
        self.dim = 2                #搜索维度    C & gamma
        self.max_iter = max_iter    #迭代次数    
        self.X = np.zeros((self.pN,self.dim))       #所有粒子的位置度    
        self.V = np.zeros((self.pN,self.dim))       #所有粒子的速度  
        self.pbest = np.zeros((self.pN,self.dim))   #个体经历的最佳位置   
        self.gbest = np.zeros((1,self.dim))         #全局最佳位置    
        self.p_fit = np.zeros(self.pN)              #每个个体的历史最佳适应值    
        self.fit = 0.650000000             #全局最佳适应值    
        
#---------------------目标函数SVM----------------------------    
    def SVM_function(self,x):    
        clf = SVC(max_iter=1000, class_weight='balanced' ,C = x[0], gamma = x[1])
        clf.fit(trainImage_CNN, trainLabels)
        acc = clf.score(testImage_CNN, testLabels)
        #print("[INFO] C = {:.10f}, gamma = {:.10f}, accuracy: {:.2f}%".format(x[0], x[1], acc * 100))
        return acc  
    
#---------------------初始化种群----------------------------------    
    def init_Population(self):    
        for i in range(self.pN):     #粒子数量
#             for j in range(self.dim):  #搜索维度
            self.X[i][0] = random.uniform(0,2)  #C  
            self.V[i][0] = random.uniform(0,0.1)  
            self.X[i][1] = random.uniform(gamma_init/2,gamma_init*2)    #gamma
            self.V[i][1] = random.uniform(gamma_init/100,gamma_init/10)    
            self.pbest[i] = self.X[i]    
            tmp = self.SVM_function(self.X[i])
            print("[INFO] {:.1f} C = {:.6f}, gamma = {:.15f}, accuracy: {:.15f}%".format(i, self.X[i][0], self.X[i][1], tmp * 100))
            self.p_fit[i] = tmp    #每个个体的历史最佳适应值 
            if(tmp > self.fit):    
                self.fit = tmp    
                self.gbest = self.X[i]    
        
#----------------------更新粒子位置----------------------------------    
    def iterator(self):    
        fitness = []    
        for t in range(self.max_iter):
            print ("[INFO] iterator: {:.0f}".format(t))
            start = time.clock()
            for i in range(self.pN):         #更新gbest\pbest    
                temp = self.SVM_function(self.X[i])    
                if(temp > self.p_fit[i]):      #更新个体最优    
                    self.p_fit[i] = temp    
                    self.pbest[i] = self.X[i]    
                    if(self.p_fit[i] > self.fit):  #更新全局最优    
                        self.gbest = self.X[i]    
                        self.fit = self.p_fit[i]    
            for i in range(self.pN):   
                tempV = self.w*self.V[i] + self.c1*self.r1*(self.pbest[i] - self.X[i]) + \
                            self.c2*self.r2*(self.gbest - self.X[i])
                tempX = self.X[i] + self.V[i]
                if (tempX[i]> 0 for i in range( self.dim)):
                    self.X[i] = self.X[i] + self.V[i] 
                else:
                    self.X[i] = self.X[i] /3 * 2
            fitness.append(self.fit) 
            print("[INFO] C = {:.6f}, gamma = {:.15f}, accuracy: {:.15f}%".format( self.gbest[0], self.gbest[1], self.fit * 100))
            elapsed = (time.clock() - start)
            print("Time used:",elapsed)
            #print(self.fit)                   #输出最优值  
            #print(self.gbest)                   #输出全局最佳位置   
        return fitness    
  
 
#当中是你的程序
 


In [8]:
my_pso = PSO(pN=8,max_iter=5)    
my_pso.init_Population()    


/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[INFO] 0.0 C = 1.252485, gamma = 0.000061239855601, accuracy: 67.721518987341767%
[INFO] 1.0 C = 1.228967, gamma = 0.000027260524112, accuracy: 70.569620253164558%
[INFO] 2.0 C = 0.260778, gamma = 0.000070916189222, accuracy: 54.430379746835442%
[INFO] 3.0 C = 0.473330, gamma = 0.000028986925951, accuracy: 70.253164556962020%
[INFO] 4.0 C = 0.868777, gamma = 0.000020548010783, accuracy: 71.202531645569621%
[INFO] 5.0 C = 0.373905, gamma = 0.000035134506496, accuracy: 66.772151898734180%
[INFO] 6.0 C = 0.895871, gamma = 0.000063056259514, accuracy: 65.189873417721529%
[INFO] 7.0 C = 0.106315, gamma = 0.000043124508261, accuracy: 59.177215189873422%


In [9]:
print (my_pso.fit)
print (my_pso.gbest)

0.7120253164556962
[8.68777360e-01 2.05480108e-05]


In [10]:
fitness = my_pso.iterator() 

[INFO] iterator: 0


/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[INFO] C = 0.931487, gamma = 0.000022660169749, accuracy: 71.202531645569621%
Time used: 913.8704370000003
[INFO] iterator: 1
[INFO] C = 0.994198, gamma = 0.000024772328714, accuracy: 71.202531645569621%
Time used: 904.7029769999999
[INFO] iterator: 2
[INFO] C = 1.056908, gamma = 0.000026884487679, accuracy: 71.202531645569621%
Time used: 1382.9503769999997
[INFO] iterator: 3
[INFO] C = 1.119618, gamma = 0.000028996646645, accuracy: 71.202531645569621%
Time used: 1403.3568850000001
[INFO] iterator: 4
[INFO] C = 1.182328, gamma = 0.000031108805610, accuracy: 71.202531645569621%
Time used: 1408.4536980000003


In [11]:
# SVC
print("\n")
print("[INFO] evaluating histogram accuracy...")
model = SVC(max_iter=1000, class_weight='balanced',C = my_pso.gbest[0],gamma = my_pso.gbest[1])
model.fit(trainImage_CNN, trainLabels)
acc = model.score(testImage_CNN, testLabels)
print("[INFO] SVM-SVC CNN accuracy: {:.10f}%".format(acc * 100))




[INFO] evaluating histogram accuracy...
[INFO] SVM-SVC CNN accuracy: 70.2531645570%
